# <ins>Products cleanup</ins>

# Import pandas and upload data.

In [1]:
import pandas as pd

In [2]:
url_brands = 'https://drive.google.com/file/d/1m1ThDDIYRTTii-rqM5SEQjJ8McidJskD/view?usp=sharing' # brands.csv
path_brands = "https://drive.google.com/uc?export=download&id="+url_brands.split("/")[-2]

brands = pd.read_csv(path_brands)

In [3]:
url_ol = 'https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing' # orderlines.csv
path_ol = "https://drive.google.com/uc?export=download&id="+url_ol.split("/")[-2]

orderlines = pd.read_csv(path_ol)

In [4]:
url_orders = 'https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing' # orders.csv
path_orders = "https://drive.google.com/uc?export=download&id="+url_orders.split("/")[-2]

orders = pd.read_csv(path_orders)

In [5]:
url_products = 'https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=drive_link' # products.csv
path_products = "https://drive.google.com/uc?export=download&id="+url_products.split("/")[-2]

products = pd.read_csv(path_products)

# Have a look at products

In [6]:
products

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364
...,...,...,...,...,...,...,...
19321,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,1,12282
19322,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392
19323,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392
19324,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,0,1392


In [7]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB


In [8]:
products.isna().sum()

sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64

# Clean up products
TODO:
* Drop duplicates √
* Drop 'promo_price' (it's a mess and apparently adds no relevant information whatsoever) √
* Drop 'in_stock' √
* Deal with the messiness of 'price' and turn into type float √
* Deal with any null values that might remain √
* Save tidy csv to drive √

## Drop duplicates
There are 8,746 duplicates (which is about 45% of the dataset)

In [9]:
# Check for duplicates
products.duplicated().sum()

8746

In [10]:
# Drop duplicates
products = products.drop_duplicates()

## Drop 'promo_price' and 'in_stock'

In [11]:
products = products.loc[:, ["sku", "name", "desc", "price", "type"]]

## Deal with the messiness of 'price' and turn into type float

### Format 'price' correctly.

#### We first have to get rid of all null values in 'price' (46 in total).

In [12]:
products = products.loc[products["price"].isna() == False]

#### How many values are formatted incorrectly?
Meaning: have two dots like for example '2.855.594'
<br>Answer: a total of 377 (3.5%).

In [13]:
wrong_price = products.loc[products["price"].str.contains('\d+\.\d+\.\d+'), "price"].tolist()
len(wrong_price)

377

In [14]:
(len(wrong_price) * 100) / products.shape[0]

3.578887412189102

In [15]:
products.loc[products["price"].str.contains('\d+\.\d+\.\d+'), "price"]

665      1.639.792
792      4.694.994
797      4.090.042
827      2.199.791
885      5.609.698
           ...    
19312    6.999.003
19313    6.999.003
19314    6.999.003
19315    6.999.003
19316    6.999.003
Name: price, Length: 377, dtype: object

Let's get rid of that first dot.

In [16]:
# Create a mask for the incorrect values
mask = products["price"].isin(wrong_price)

# Use the mask to remove the first period in incorrect unit prices
products.loc[mask, "price"] = products.loc[mask, "price"].str.replace('\.', '', 1, regex=True)

<ipython-input-16-d91dfffb49ad>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products.loc[mask, "price"] = products.loc[mask, "price"].str.replace('\.', '', 1, regex=True)


Compare with first three values in output above.

In [17]:
products.loc[[665, 792, 797], "price"]

665    1639.792
792    4694.994
797    4090.042
Name: price, dtype: object

#### Set 'price' to type float.

In [18]:
products["price"] =  pd.to_numeric(products["price"])

## Deal with any null values that might remain
There are some in 'desc' and 'type'. Let's keep them.

In [19]:
products.isna().sum()

sku       0
name      0
desc      7
price     0
type     50
dtype: int64

In [ ]:
products

## Save tidy csv to drive

In [ ]:
from google.colab import drive

drive.mount('drive', force_remount=True)
products.to_csv('/content/drive/My Drive/Bootcamp/03 data cleaning/data/products_tidy.csv')

Mounted at drive
